# Villin simple design
Here we attempt to do the simplest possible modelling, which just tries to model new side chains at random that improve the total energy. We use the simple villin headpiece subdomain as an example. Here, the specifics for repeat proteins are not required. We use a favourite protein as a toy model.

In [16]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from repeatdesigner import designer as rd

In [18]:
helical = [4,5,6,7,8,15,16,17,22,23,24,25,26,27,28,29,30,31]
villin_des = rd.Design(pdb="pdbs/1vii.pdb", targets=helical)

 Generated new protein design 
 .. name : pdbs/1vii
 .. file : pdbs/1vii.pdb
 .. sequence : MLSDEDFKAVFGMTRSAFANLPLWKQQNLKKEKGLF
 .. target residues :  [4, 5, 6, 7, 8, 15, 16, 17, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [19]:
mc_villin = rd.Optimizer(villin_des, beta=0.01, len_mc=100, nruns=10, energy='global')

In [ ]:
mc_villin.run_mc()

In [ ]:
fig, ax = plt.subplots()
for k,v in mc_villin.models.iteritems():
    ax.plot(v['score'])

ax.set_ylabel('DOPE energy', fontsize=14)
ax.set_xlabel('MC steps', fontsize=14)

In [47]:
import Bio.PDB
import Bio.Seq
import Bio.SeqUtils
import Bio.pairwise2
import Bio.SeqRecord
import Bio.Align.AlignInfo
for k,v in mc_villin.models.iteritems():
    print "%3i %10.2f %s"%(k, v['score'][-1][0], \
        Bio.Seq.Seq(''.join([Bio.SeqUtils.seq1(x.get_resname()) \
                    for x in v['model'].get_residues()])))

  0   -3236.22 MLSRTEWVAVFGMTWHAFANLKPTWYCKWTHEKGLF
  1   -3497.37 MLSIELWVAVFGMTYFEFANLRFWDPSQLYFEKGLF
  2   -3058.84 MLSWLPSFAVFGMTCWRFANLTLRHATEWNTEKGLF
  3   -3125.24 MLSCVQTMAVFGMTCNLFANLDCPEYIPFLCEKGLF
  4   -3701.52 MLSMRYMLAVFGMTFCNFANLPKFVCLILVYEKGLF
  5   -3161.66 MLSWMVHWAVFGMTPIMFANLFCRIQKFVMCEKGLF
  6   -3584.51 MLSQPFIYAVFGMTWYWFANLDFWWLTKTCREKGLF
  7   -3895.13 MLSLNSFLAVFGMTFTLFANLPHCRWLYVFWEKGLF
  8   -3372.93 MLSDNERWAVFGMTMLYFANLFYFWADYCLIEKGLF
  9   -3288.98 MLSCCYLWAVFGMTSTKFANLGTRIWEWPFKEKGLF
 10   -3274.97 MLSLCYVKAVFGMTFKHFANLTIPKVKGLMVEKGLF
 11   -2815.23 MLSPKLTYAVFGMTETWFANLHWVMEFKRQPEKGLF
 12   -3432.83 MLSGTLRHAVFGMTAYKFANLFLWLLFWWFKEKGLF
 13   -2987.80 MLSEFFQAAVFGMTRSRFANLFWKFYYLIGDEKGLF
 14   -3396.19 MLSSGMEYAVFGMTIKYFANLAHCYIFRYIFEKGLF
 15   -3447.30 MLSCKFLQAVFGMTVWSFANLDPVQRYEVLWEKGLF
 16   -3370.82 MLSYGVSWAVFGMTRLWFANLMYWKIMICIHEKGLF
 17   -2841.35 MLSPWNSHAVFGMTDTEFANLFQAVERDYTFEKGLF
 18   -3332.18 MLSVAWWVAVFGMTKNYFANLVWDWWIRLNLEKGLF
 19   -3181.

In [51]:
sequences = [Bio.SeqRecord.SeqRecord(Bio.Seq.Seq(''.join([Bio.SeqUtils.seq1(x.get_resname()) \
                    for x in v['model'].get_residues()]))) for k, v in mc_villin.models.iteritems() if v['score'][-1][0]<-3200.]
align =  Bio.Align.MultipleSeqAlignment(sequences)
summary_align = Bio.Align.AlignInfo.SummaryInfo(align)
print villin_des.seq
print summary_align.dumb_consensus(threshold=(0.1))
print summary_align.dumb_consensus(threshold=(0.2))
print summary_align.dumb_consensus(threshold=(0.5))


MLSDEDFKAVFGMTRSAFANLPLWKQQNLKKEKGLF
MLSMLWMWAVFGMTXXYFANLXFWFWFKLFFEKGLF
MLSXXXXXAVFGMTXXXFANLXXXXXXXLFXEKGLF
MLSXXXXXAVFGMTXXXFANLXXXXXXXXXXEKGLF


In [53]:
len(sequences)*0.2

15.8